In [1]:
import numpy as np
import pandas as pd
import os
import yaml

#### Paths & Vars

In [2]:
# vars
adapt_methods = ['coral','al_layer']
gen_methods = ['irm','dro','coral','al_layer']
tasks = ['mortality','longlos','invasivevent','sepsis']
metrics = ['auc','auprc','ace_abs_logistic_log']
Ns = [100,500,1000,1500]

# paths
tables_fpath = '/hpf/projects/lsung/projects/mimic4ds/Experiments/domain_adapt/tables/'
adapt_fpath = '/hpf/projects/lsung/projects/mimic4ds/Experiments/domain_adapt'
gen_fpath = '/hpf/projects/lsung/projects/mimic4ds/Experiments/domain_gen'
base_fpath = '/hpf/projects/lsung/projects/mimic4ds/Experiments/baseline'

### Collect OOD Results

In [3]:
alpha = 0.05

results = {
    "base":{
        "eval":pd.read_csv(f"{base_fpath}/results/model_evaluation_{alpha}.csv").query("phase=='test'"),
        "compare":pd.read_csv(f"{base_fpath}/results/model_comparison_{alpha}.csv").query("phase=='test'"),
    },
    "gen":{
        "eval":pd.read_csv(f"{gen_fpath}/results/model_evaluation_{alpha}.csv").query("phase=='test' and `lambda`==-1 and group==1"),
        "compare":pd.read_csv(f"{gen_fpath}/results/model_comparison_{alpha}.csv").query("phase=='test' and `lambda`==-1 and group==1"),
    },
    "adapt":{
        "eval":pd.read_csv(f"{adapt_fpath}/results/model_evaluation_{alpha}.csv").query("phase=='test' and `lambda`==-1 and group==1"),
        "compare":pd.read_csv(f"{adapt_fpath}/results/model_comparison_{alpha}.csv").query("phase=='test' and `lambda`==-1 and group==1"),
    },
}

df_results_all = {}
for eval_method in ['avg', 'ensemble', 'best']:
    # base results
    df_base = results['base']['eval'].query(
        "\
        train_group=='2008 - 2010' and\
        eval_group=='2017 - 2019' and\
        evaluation_method==@eval_method and\
        metric==@metrics\
        "
    )[['analysis_id', 'metric', 'ci_lower','ci_med','ci_upper']]
    df_base['group']=1
    df_base['train_method']='Baseline [08-10]'

    # oracle results
    df_oracle = results['base']['eval'].query(
        "\
        train_group=='2017 - 2019' and\
        eval_group=='2017 - 2019' and\
        evaluation_method==@eval_method and\
        metric==@metrics\
        "
    )[['analysis_id', 'metric', 'ci_lower','ci_med','ci_upper']]
    df_oracle['group']=1
    df_oracle['train_method'] = 'Oracle [17-19]'

    # ERM results
    df_erm = results['gen']['eval'].query(
        "\
        train_method=='erm' and\
        evaluation_method==@eval_method and\
        metric==@metrics\
        "
    )[['analysis_id', 'metric', 'ci_lower','ci_med','ci_upper','group','train_method']]

    # domain gen results
    df_gen = results['gen']['eval'].query(
        "\
        train_method!='erm' and\
        evaluation_method==@eval_method and\
        metric==@metrics\
        "
    )[['analysis_id', 'metric', 'ci_lower','ci_med','ci_upper','group','train_method']]
    df_gen['framework'] = 'Domain Generalization'

    stats = results['gen']['compare'].query(
        "\
        evaluation_method==@eval_method and\
        metric==@metrics\
        "
    )[['analysis_id', 'metric', 'ci_lower','ci_upper','group','train_method']]
    stats['sig'] = stats['ci_lower']*stats['ci_upper']>0
    stats.drop(columns=['ci_lower','ci_upper'],inplace=True)

    df_gen = pd.merge(
        df_gen, 
        stats, 
        how='left', 
        left_on=['analysis_id','metric','group', 'train_method'],
        right_on = ['analysis_id','metric','group','train_method']
    )

    # domain adapt results
    df_adapt = results['adapt']['eval'].query(
        "\
        evaluation_method==@eval_method and\
        metric==@metrics\
        "
    )[['analysis_id', 'metric', 'ci_lower','ci_med','ci_upper','group','train_method','n_ood']]
    df_adapt['framework'] = 'Domain Adaptation'
    
    stats = results['adapt']['compare'].query(
        "\
        evaluation_method==@eval_method and\
        metric==@metrics\
        "
    )[['analysis_id', 'metric', 'ci_lower','ci_upper','group','train_method','n_ood']]
    stats['sig'] = stats['ci_lower']*stats['ci_upper']>0
    stats.drop(columns=['ci_lower','ci_upper'],inplace=True)

    df_adapt = pd.merge(
        df_adapt, 
        stats, 
        how='left', 
        left_on=['analysis_id','metric','group', 'train_method','n_ood'],
        right_on = ['analysis_id','metric','group','train_method','n_ood']
    )
    
    # join results
    df_results = pd.concat((df_base, df_oracle, df_erm, df_gen, df_adapt))
    df_results['sig'].replace({
        False:'',
        True:'*',
        np.nan:''
    }, inplace=True)

    # combine CI columns into performance column
    df_results['Performance'] = (
        df_results['ci_med'].apply('{:.3f}'.format) + 
        df_results['sig'] +
        " (" + 
        df_results['ci_lower'].apply('{:.3f}'.format) + 
        ',' + 
        df_results['ci_upper'].apply('{:.3f}'.format) +
        ')'
    )
    df_results.drop(columns=['ci_lower','ci_med','ci_upper'],inplace=True)
    df_results.fillna(" ",inplace=True)

    # rename columns and values
    df_results.rename(columns = {
        'metric':'Metric',
        'group':'Year Group',
        'analysis_id':'Task',
        'n_ood':'Unlabeled OOD Samples',
        'framework': 'Framework',
        'train_method':'Method'},inplace=True)

    # Replace values
    df_results['Year Group'].replace(
        {
            0:"2008 - 2016 [ID]",
            1:"2017 - 2019 [OOD]"
        },
        inplace=True
    )
    df_results['Metric'].replace(
        {
            'auc':"AUROC",
            'auprc':"AUPRC",
            'ace_abs_logistic_log':'Calibration'
        },
        inplace=True
    )
    df_results['Method'].replace(
        {
            'al_layer':"AL",
            'coral':"CORAL",
            'erm':"ERM",
            'irm':'IRM',
            'dro':'GroupDRO'
        },
        inplace=True
    )
    df_results['Task'].replace(
        {
            'longlos':'Long LOS',
            'sepsis':'Sepsis',
            'mortality':'Mortality',
            'invasivevent':'Invasive Ventilation',
        },
        inplace=True
    )

    # pivot table    
    df_results = df_results.pivot(index=["Task","Year Group","Metric"],columns=["Framework","Unlabeled OOD Samples","Method"],values=["Performance"])
    df_results.fillna(" ",inplace = True)
    df_results.columns = pd.MultiIndex.from_tuples([x[1:] for x in df_results.columns], names = ['Framework','Unlabelled OOD Samples','Method'])
    
    # Order Indices & Columns
    df_results = df_results[[
        (                    ' ',    ' ', 'Baseline [08-10]'),
        (                    ' ',    ' ',   'Oracle [17-19]'),
        (                    ' ',    ' ',              'ERM'),
        ('Domain Generalization',    ' ',              'IRM'),
        ('Domain Generalization',    ' ',         'GroupDRO'),
        ('Domain Generalization',    ' ',               'AL'),
        ('Domain Generalization',    ' ',            'CORAL'),
        (    'Domain Adaptation',  100.0,               'AL'),
        (    'Domain Adaptation',  100.0,            'CORAL'),
        (    'Domain Adaptation',  500.0,               'AL'),
        (    'Domain Adaptation',  500.0,            'CORAL'),
        (    'Domain Adaptation', 1000.0,               'AL'),
        (    'Domain Adaptation', 1000.0,            'CORAL'),
        (    'Domain Adaptation', 1500.0,               'AL'),
        (    'Domain Adaptation', 1500.0,            'CORAL'),
    ]]
    
    df_results = df_results.reindex(labels = ['Long LOS','Sepsis','Mortality','Invasive Ventilation'], level=0)
    df_results = df_results.reindex(labels = ['AUROC','AUPRC','Calibration'],level=2)
    #df_results = df_results.style.apply(highlight_sig, axis=0)
    
    # add to dictionary
    df_results_all[eval_method] = df_results

### Comparison of Average Model Performance

In [4]:
df_results_all['avg']

Framework                                                                \
Unlabelled OOD Samples                                                    
Method                                                 Baseline [08-10]   
Task                 Year Group        Metric                             
Long LOS             2017 - 2019 [OOD] AUROC        0.644 (0.637,0.650)   
                                       AUPRC        0.512 (0.501,0.522)   
                                       Calibration  0.089 (0.084,0.093)   
Sepsis               2017 - 2019 [OOD] AUROC        0.645 (0.634,0.656)   
                                       AUPRC        0.155 (0.148,0.164)   
                                       Calibration  0.065 (0.061,0.068)   
Mortality            2017 - 2019 [OOD] AUROC        0.877 (0.869,0.885)   
                                       AUPRC        0.464 (0.443,0.486)   
                                       Calibration  0.014 (0.013,0.016)   
Invasive Ventilation 2017 - 2019 [OOD] AUROC        0.878 (0.870,0.886)   
                                       AUPRC        0.639 (0.619,0.659)   
                                       Calibration  0.025 (0.023,0.027)   

Framework                                                                \
Unlabelled OOD Samples                                                    
Method                                                   Oracle [17-19]   
Task                 Year Group        Metric                             
Long LOS             2017 - 2019 [OOD] AUROC        0.702 (0.696,0.708)   
                                       AUPRC        0.542 (0.532,0.552)   
                                       Calibration  0.032 (0.029,0.036)   
Sepsis               2017 - 2019 [OOD] AUROC        0.735 (0.724,0.746)   
                                       AUPRC        0.298 (0.280,0.317)   
                                       Calibration  0.013 (0.011,0.016)   
Mortality            2017 - 2019 [OOD] AUROC        0.903 (0.897,0.909)   
                                       AUPRC        0.494 (0.472,0.515)   
                                       Calibration  0.014 (0.012,0.015)   
Invasive Ventilation 2017 - 2019 [OOD] AUROC        0.880 (0.872,0.888)   
                                       AUPRC        0.622 (0.601,0.642)   
                                       Calibration  0.020 (0.019,0.022)   

Framework                                                                \
Unlabelled OOD Samples                                                    
Method                                                              ERM   
Task                 Year Group        Metric                             
Long LOS             2017 - 2019 [OOD] AUROC        0.678 (0.671,0.684)   
                                       AUPRC        0.533 (0.522,0.543)   
                                       Calibration  0.061 (0.057,0.065)   
Sepsis               2017 - 2019 [OOD] AUROC        0.694 (0.683,0.705)   
                                       AUPRC        0.195 (0.184,0.206)   
                                       Calibration  0.031 (0.028,0.034)   
Mortality            2017 - 2019 [OOD] AUROC        0.894 (0.887,0.901)   
                                       AUPRC        0.541 (0.520,0.562)   
                                       Calibration  0.018 (0.016,0.020)   
Invasive Ventilation 2017 - 2019 [OOD] AUROC        0.885 (0.877,0.893)   
                                       AUPRC        0.647 (0.627,0.666)   
                                       Calibration  0.019 (0.018,0.020)   

Framework                                          Domain Generalization  \
Unlabelled OOD Samples                                                     
Method                                                               IRM   
Task                 Year Group        Metric                              
Long LOS             2017 - 2019 [OOD] AUROC         0.678 (0.671,0.684)   
                 

In [6]:
df_results_all['best']

Framework                                                                \
Unlabelled OOD Samples                                                    
Method                                                 Baseline [08-10]   
Task                 Year Group        Metric                             
Long LOS             2017 - 2019 [OOD] AUROC        0.650 (0.621,0.679)   
                                       AUPRC        0.513 (0.467,0.559)   
                                       Calibration  0.074 (0.054,0.093)   
Sepsis               2017 - 2019 [OOD] AUROC        0.639 (0.588,0.689)   
                                       AUPRC        0.154 (0.120,0.194)   
                                       Calibration  0.063 (0.048,0.079)   
Mortality            2017 - 2019 [OOD] AUROC        0.874 (0.835,0.907)   
                                       AUPRC        0.456 (0.359,0.548)   
                                       Calibration  0.014 (0.008,0.024)   
Invasive Ventilation 2017 - 2019 [OOD] AUROC        0.890 (0.854,0.922)   
                                       AUPRC        0.641 (0.545,0.727)   
                                       Calibration  0.025 (0.019,0.034)   

Framework                                                                \
Unlabelled OOD Samples                                                    
Method                                                   Oracle [17-19]   
Task                 Year Group        Metric                             
Long LOS             2017 - 2019 [OOD] AUROC        0.701 (0.673,0.727)   
                                       AUPRC        0.539 (0.494,0.584)   
                                       Calibration  0.023 (0.017,0.039)   
Sepsis               2017 - 2019 [OOD] AUROC        0.754 (0.705,0.800)   
                                       AUPRC        0.308 (0.229,0.392)   
                                       Calibration  0.012 (0.005,0.027)   
Mortality            2017 - 2019 [OOD] AUROC        0.904 (0.874,0.929)   
                                       AUPRC        0.490 (0.391,0.584)   
                                       Calibration  0.015 (0.009,0.024)   
Invasive Ventilation 2017 - 2019 [OOD] AUROC        0.878 (0.840,0.911)   
                                       AUPRC        0.624 (0.529,0.707)   
                                       Calibration  0.018 (0.014,0.029)   

Framework                                                                \
Unlabelled OOD Samples                                                    
Method                                                              ERM   
Task                 Year Group        Metric                             
Long LOS             2017 - 2019 [OOD] AUROC        0.676 (0.648,0.704)   
                                       AUPRC        0.531 (0.486,0.577)   
                                       Calibration  0.057 (0.038,0.077)   
Sepsis               2017 - 2019 [OOD] AUROC        0.701 (0.650,0.749)   
                                       AUPRC        0.201 (0.155,0.256)   
                                       Calibration  0.034 (0.019,0.050)   
Mortality            2017 - 2019 [OOD] AUROC        0.895 (0.861,0.924)   
                                       AUPRC        0.539 (0.440,0.631)   
                                       Calibration  0.018 (0.010,0.028)   
Invasive Ventilation 2017 - 2019 [OOD] AUROC        0.885 (0.847,0.918)   
                                       AUPRC        0.653 (0.560,0.735)   
                                       Calibration  0.020 (0.017,0.027)   

Framework                                          Domain Generalization  \
Unlabelled OOD Samples                                                     
Method                                                               IRM   
Task                 Year Group        Metric                              
Long LOS             2017 - 2019 [OOD] AUROC         0.681 (0.653,0.709)   
                 

## Collect ID Results

In [8]:
results = {
    "gen":{
        "eval":pd.read_csv(f"{gen_fpath}/results/model_evaluation_{alpha}.csv").query("phase=='test' and `lambda`==-1 and group==0"),
        "compare":pd.read_csv(f"{gen_fpath}/results/model_comparison_{alpha}.csv").query("phase=='test' and `lambda`==-1 and group==0"),
    },
    "adapt":{
        "eval":pd.read_csv(f"{adapt_fpath}/results/model_evaluation_{alpha}.csv").query("phase=='test' and `lambda`==-1 and group==0"),
        "compare":pd.read_csv(f"{adapt_fpath}/results/model_comparison_{alpha}.csv").query("phase=='test' and `lambda`==-1 and group==0"),
    },
}

df_results_all = {}
for eval_method in ['avg', 'ensemble', 'best']:
    # ERM results
    df_erm = results['gen']['eval'].query(
        "\
        train_method=='erm' and\
        evaluation_method==@eval_method and\
        metric==@metrics\
        "
    )[['analysis_id', 'metric', 'ci_lower','ci_med','ci_upper','group','train_method']]

    # domain gen results
    df_gen = results['gen']['eval'].query(
        "\
        train_method!='erm' and\
        evaluation_method==@eval_method and\
        metric==@metrics\
        "
    )[['analysis_id', 'metric', 'ci_lower','ci_med','ci_upper','group','train_method']]
    df_gen['framework'] = 'Domain Generalization'

    stats = results['gen']['compare'].query(
        "\
        evaluation_method==@eval_method and\
        metric==@metrics\
        "
    )[['analysis_id', 'metric', 'ci_lower','ci_upper','group','train_method']]
    stats['sig'] = stats['ci_lower']*stats['ci_upper']>0
    stats.drop(columns=['ci_lower','ci_upper'],inplace=True)

    df_gen = pd.merge(
        df_gen, 
        stats, 
        how='left', 
        left_on=['analysis_id','metric','group', 'train_method'],
        right_on = ['analysis_id','metric','group','train_method']
    )

    # domain adapt results
    df_adapt = results['adapt']['eval'].query(
        "\
        evaluation_method==@eval_method and\
        metric==@metrics\
        "
    )[['analysis_id', 'metric', 'ci_lower','ci_med','ci_upper','group','train_method','n_ood']]
    df_adapt['framework'] = 'Domain Adaptation'
    
    stats = results['adapt']['compare'].query(
        "\
        evaluation_method==@eval_method and\
        metric==@metrics\
        "
    )[['analysis_id', 'metric', 'ci_lower','ci_upper','group','train_method','n_ood']]
    stats['sig'] = stats['ci_lower']*stats['ci_upper']>0
    stats.drop(columns=['ci_lower','ci_upper'],inplace=True)

    df_adapt = pd.merge(
        df_adapt, 
        stats, 
        how='left', 
        left_on=['analysis_id','metric','group', 'train_method','n_ood'],
        right_on = ['analysis_id','metric','group','train_method','n_ood']
    )
    
    # join results
    df_results = pd.concat((df_erm, df_gen, df_adapt))
    df_results['sig'].replace({
        False:'',
        True:'*',
        np.nan:''
    }, inplace=True)

    # combine CI columns into performance column
    df_results['Performance'] = (
        df_results['ci_med'].apply('{:.3f}'.format) + 
        df_results['sig'] +
        " (" + 
        df_results['ci_lower'].apply('{:.3f}'.format) + 
        ',' + 
        df_results['ci_upper'].apply('{:.3f}'.format) +
        ')'
    )
    df_results.drop(columns=['ci_lower','ci_med','ci_upper'],inplace=True)
    df_results.fillna(" ",inplace=True)

    # rename columns and values
    df_results.rename(columns = {
        'metric':'Metric',
        'group':'Year Group',
        'analysis_id':'Task',
        'n_ood':'Unlabeled OOD Samples',
        'framework': 'Framework',
        'train_method':'Method'},inplace=True)

    # Replace values
    df_results['Year Group'].replace(
        {
            0:"2008 - 2016 [ID]",
            1:"2017 - 2019 [OOD]"
        },
        inplace=True
    )
    df_results['Metric'].replace(
        {
            'auc':"AUROC",
            'auprc':"AUPRC",
            'ace_abs_logistic_log':'Calibration'
        },
        inplace=True
    )
    df_results['Method'].replace(
        {
            'al_layer':"AL",
            'coral':"CORAL",
            'erm':"ERM",
            'irm':'IRM',
            'dro':'GroupDRO'
        },
        inplace=True
    )
    df_results['Task'].replace(
        {
            'longlos':'Long LOS',
            'sepsis':'Sepsis',
            'mortality':'Mortality',
            'invasivevent':'Invasive Ventilation',
        },
        inplace=True
    )

    # pivot table    
    df_results = df_results.pivot(index=["Task","Year Group","Metric"],columns=["Framework","Unlabeled OOD Samples","Method"],values=["Performance"])
    df_results.fillna(" ",inplace = True)
    df_results.columns = pd.MultiIndex.from_tuples([x[1:] for x in df_results.columns], names = ['Framework','Unlabelled OOD Samples','Method'])
    
    # Order Indices & Columns
    df_results = df_results[[
        (                    ' ',    ' ',              'ERM'),
        ('Domain Generalization',    ' ',              'IRM'),
        ('Domain Generalization',    ' ',         'GroupDRO'),
        ('Domain Generalization',    ' ',               'AL'),
        ('Domain Generalization',    ' ',            'CORAL'),
        (    'Domain Adaptation',  100.0,               'AL'),
        (    'Domain Adaptation',  100.0,            'CORAL'),
        (    'Domain Adaptation',  500.0,               'AL'),
        (    'Domain Adaptation',  500.0,            'CORAL'),
        (    'Domain Adaptation', 1000.0,               'AL'),
        (    'Domain Adaptation', 1000.0,            'CORAL'),
        (    'Domain Adaptation', 1500.0,               'AL'),
        (    'Domain Adaptation', 1500.0,            'CORAL'),
    ]]
    
    df_results = df_results.reindex(labels = ['Long LOS','Sepsis','Mortality','Invasive Ventilation'], level=0)
    df_results = df_results.reindex(labels = ['AUROC','AUPRC','Calibration'],level=2)
    #df_results = df_results.style.apply(highlight_sig, axis=0)
    
    # add to dictionary
    df_results_all[eval_method] = df_results

In [9]:
df_results_all['avg']

Framework                                                               \
Unlabelled OOD Samples                                                   
Method                                                             ERM   
Task                 Year Group       Metric                             
Long LOS             2008 - 2016 [ID] AUROC        0.744 (0.741,0.748)   
                                      AUPRC        0.557 (0.550,0.563)   
                                      Calibration  0.022 (0.021,0.023)   
Sepsis               2008 - 2016 [ID] AUROC        0.815 (0.810,0.821)   
                                      AUPRC        0.423 (0.410,0.436)   
                                      Calibration  0.013 (0.011,0.014)   
Mortality            2008 - 2016 [ID] AUROC        0.883 (0.879,0.887)   
                                      AUPRC        0.425 (0.412,0.438)   
                                      Calibration  0.016 (0.015,0.018)   
Invasive Ventilation 2008 - 2016 [ID] AUROC        0.844 (0.839,0.850)   
                                      AUPRC        0.585 (0.572,0.597)   
                                      Calibration  0.018 (0.017,0.019)   

Framework                                         Domain Generalization  \
Unlabelled OOD Samples                                                    
Method                                                              IRM   
Task                 Year Group       Metric                              
Long LOS             2008 - 2016 [ID] AUROC        0.743* (0.740,0.747)   
                                      AUPRC        0.554* (0.548,0.561)   
                                      Calibration   0.022 (0.021,0.024)   
Sepsis               2008 - 2016 [ID] AUROC         0.815 (0.810,0.821)   
                                      AUPRC         0.426 (0.412,0.439)   
                                      Calibration   0.013 (0.012,0.015)   
Mortality            2008 - 2016 [ID] AUROC         0.883 (0.879,0.887)   
                                      AUPRC         0.425 (0.412,0.438)   
                                      Calibration  0.015* (0.014,0.017)   
Invasive Ventilation 2008 - 2016 [ID] AUROC         0.844 (0.838,0.849)   
                                      AUPRC         0.583 (0.571,0.595)   
                                      Calibration   0.018 (0.017,0.020)   

Framework                                                                \
Unlabelled OOD Samples                                                    
Method                                                         GroupDRO   
Task                 Year Group       Metric                              
Long LOS             2008 - 2016 [ID] AUROC        0.741* (0.738,0.745)   
                                      AUPRC        0.554* (0.548,0.561)   
                                      Calibration  0.029* (0.028,0.031)   
Sepsis               2008 - 2016 [ID] AUROC        0.803* (0.797,0.809)   
                                      AUPRC        0.391* (0.378,0.404)   
                                      Calibration  0.019* (0.017,0.021)   
Mortality            2008 - 2016 [ID] AUROC        0.852* (0.846,0.857)   
                                      AUPRC        0.415* (0.402,0.428)   
                                      Calibration  0.044* (0.042,0.045)   
Invasive Ventilation 2008 - 2016 [ID] AUROC         0.844 (0.838,0.849)   
                                      AUPRC         0.581 (0.569,0.594)   
                                      Calibration  0.025* (0.023,0.027)   

Framework                                                                \
Unlabelled OOD Samples                                                    
Method                                                               AL   
Task                 Year Group       Metric                              
Long LOS             2008 - 2016 [ID] AUROC         0.744 (0.740,0.747)   
                                      

In [10]:
df_results_all['best']

Framework                                                               \
Unlabelled OOD Samples                                                   
Method                                                             ERM   
Task                 Year Group       Metric                             
Long LOS             2008 - 2016 [ID] AUROC        0.746 (0.730,0.762)   
                                      AUPRC        0.560 (0.531,0.589)   
                                      Calibration  0.020 (0.017,0.025)   
Sepsis               2008 - 2016 [ID] AUROC        0.814 (0.787,0.840)   
                                      AUPRC        0.432 (0.372,0.492)   
                                      Calibration  0.008 (0.006,0.016)   
Mortality            2008 - 2016 [ID] AUROC        0.883 (0.863,0.902)   
                                      AUPRC        0.420 (0.363,0.477)   
                                      Calibration  0.017 (0.011,0.022)   
Invasive Ventilation 2008 - 2016 [ID] AUROC        0.846 (0.822,0.870)   
                                      AUPRC        0.599 (0.545,0.648)   
                                      Calibration  0.016 (0.015,0.021)   

Framework                                         Domain Generalization  \
Unlabelled OOD Samples                                                    
Method                                                              IRM   
Task                 Year Group       Metric                              
Long LOS             2008 - 2016 [ID] AUROC         0.742 (0.726,0.758)   
                                      AUPRC         0.554 (0.524,0.583)   
                                      Calibration   0.024 (0.020,0.036)   
Sepsis               2008 - 2016 [ID] AUROC         0.819 (0.793,0.845)   
                                      AUPRC         0.427 (0.367,0.489)   
                                      Calibration   0.014 (0.006,0.023)   
Mortality            2008 - 2016 [ID] AUROC         0.883 (0.863,0.902)   
                                      AUPRC         0.428 (0.370,0.486)   
                                      Calibration   0.016 (0.010,0.022)   
Invasive Ventilation 2008 - 2016 [ID] AUROC         0.845 (0.820,0.869)   
                                      AUPRC         0.581 (0.524,0.635)   
                                      Calibration   0.018 (0.015,0.025)   

Framework                                                                \
Unlabelled OOD Samples                                                    
Method                                                         GroupDRO   
Task                 Year Group       Metric                              
Long LOS             2008 - 2016 [ID] AUROC         0.741 (0.725,0.757)   
                                      AUPRC         0.557 (0.527,0.585)   
                                      Calibration   0.026 (0.021,0.035)   
Sepsis               2008 - 2016 [ID] AUROC         0.804 (0.775,0.831)   
                                      AUPRC         0.399 (0.341,0.458)   
                                      Calibration   0.010 (0.006,0.019)   
Mortality            2008 - 2016 [ID] AUROC        0.860* (0.836,0.882)   
                                      AUPRC         0.425 (0.366,0.482)   
                                      Calibration  0.037* (0.030,0.044)   
Invasive Ventilation 2008 - 2016 [ID] AUROC         0.847 (0.822,0.871)   
                                      AUPRC         0.587 (0.530,0.640)   
                                      Calibration  0.025* (0.017,0.033)   

Framework                                                               \
Unlabelled OOD Samples                                                   
Method                                                              AL   
Task                 Year Group       Metric                             
Long LOS             2008 - 2016 [ID] AUROC        0.741 (0.725,0.758)   
                                      AUPRC